In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
from ipywidgets import interact, FloatSlider

In [ ]:
alpha = 0.1
gamma = 1.0
t_span = (0, 20)
t_eval = np.linspace(t_span[0], t_span[1], 1000)

In [ ]:
def system_A(t, N, alpha, gamma):
    x, y = N
    dx = x - x*y/(1 + alpha*x)
    dy = -gamma*y + x*y/(1 + alpha*x)
    return [dx, dy]

In [ ]:
def plot_model_A(alpha=0.1, gamma=1.0):
    t_span = (0, 50)
    t_eval = np.linspace(*t_span, 1000)

    N0 = [1.5, 0.8]
    sol = solve_ivp(system_A, t_span, N0, t_eval=t_eval, args=(alpha, gamma))

    plt.figure(figsize=(12,5))

    # (1) часова динаміка
    plt.subplot(1,2,1)
    plt.plot(sol.t, sol.y[0], label='x (жертви)')
    plt.plot(sol.t, sol.y[1], label='y (хижаки)')
    plt.xlabel('t'); plt.ylabel('Популяції')
    plt.title(f'Модель A: α={alpha}, γ={gamma}')
    plt.legend(); plt.grid(True)

    # (2) фазовий портрет
    plt.subplot(1,2,2)
    x_vals = np.linspace(0, 5, 25)
    y_vals = np.linspace(0, 5, 25)
    X, Y = np.meshgrid(x_vals, y_vals)

    U = X - (X*Y)/(1 + alpha*X)
    V = -gamma*Y + (X*Y)/(1 + alpha*X)

    plt.streamplot(X, Y, U, V, density=1.2, linewidth=1, arrowsize=1, color='black')

    # кілька траєкторій з різних початкових умов
    initials = [[0.5, 0.5], [1, 2], [2, 1], [3, 3], [4, 1]]
    for N0 in initials:
        sol = solve_ivp(system_A, t_span, N0, t_eval=t_eval, args=(alpha, gamma))
        plt.plot(sol.y[0], sol.y[1], lw=2)

    plt.xlabel('x (жертви)')
    plt.ylabel('y (хижаки)')
    plt.title('Фазовий портрет')
    plt.xlim(0, 5)
    plt.ylim(0, 5)
    plt.grid(True)

    plt.tight_layout()
    plt.show()

interact(plot_model_A,gamma=FloatSlider(min=0, max=2.0, step=0.01, value=1.0),
        alpha=FloatSlider(min=0, max=2, step=0.01, value=1.0))
         #alpha=FloatSlider(min=1/gamma, max=1+gamma, step=0.1, value=1.0))

interactive(children=(FloatSlider(value=1.0, description='alpha', max=2.0, step=0.01), FloatSlider(value=1.0, …

<function __main__.plot_model_A(alpha=0.1, gamma=1.0)>

# Особливі точки


In [ ]:
alpha = 0.1
gamma = 1.0

def system_A_eqs(N, alpha, gamma):
    x, y = N
    dx = x - x*y/(1 + alpha*x)
    dy = -gamma*y + x*y/(1 + alpha*x)
    return [dx, dy]
# Якобі
def jacobian_A(x, y, alpha, gamma):
    dfdx = 1 - y*(1 + alpha*x - alpha*x)/(1 + alpha*x)**2 - y/(1 + alpha*x)**2 * 0  # скоротимо нижче
    dfdx = 1 - (y / (1 + alpha*x)**2)
    dfdy = -x / (1 + alpha*x)
    dgdx = y / (1 + alpha*x)**2
    dgdy = -gamma + x / (1 + alpha*x)
    return np.array([[dfdx, dfdy],
                     [dgdx, dgdy]])
# список різних початкових наближень, щоб знайти усі можливі точки
guesses = [
    [0.0, 0.0],
    [0.1, 0.1],
    [0.5, 0.1],
    [0.1, 0.5],
    [0.8, 0.8],
    [1.0, 0.1],
    [0.1, 1.0],
    [1.0, 1.0]
]

solutions = []
for guess in guesses:
    sol = fsolve(system_A_eqs, guess, args=(alpha, gamma))
    sol = np.round(sol, 5)
    if all(sol >= 0):
        if not any(np.allclose(sol, s, atol=1e-4) for s in solutions):
            solutions.append(sol)

print("Особливі точки та їх типи:")
for s in solutions:
    x, y = s
    J = jacobian_A(x, y, alpha, gamma)
    eigvals = np.linalg.eigvals(J)
    real_parts = np.real(eigvals)

    print(f"\nТочка ({x:.4f}, {y:.4f})")
    print(f"  Власні числа: {eigvals}")

    if np.all(real_parts < 0):
        print("  → стійкий вузол або фокус")
    elif np.all(real_parts > 0):
        print("  → нестійкий вузол або фокус")
    elif real_parts[0] * real_parts[1] < 0:
        print("  → сідлова точка")
    else:
        print("  → центр або нейтральна стабільність")

Особливі точки та їх типи:

Точка (0.0000, 0.0000)
  Власні числа: [ 1. -1.]
  → сідлова точка

Точка (1.1111, 1.1111)
  Власні числа: [0.04999991+0.94736392j 0.04999991-0.94736392j]
  → нестійкий вузол або фокус


In [ ]:
%reset -f

In [ ]:
#B

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
from ipywidgets import interact, FloatSlider

In [ ]:
def system_B(t, N, gamma, epsilon1, epsilon2, l1, l2):
    x, y = N
    dx = x*(x - l1)*(1 - x) - epsilon1*x*y
    dy = gamma*y*((y - l2)*(1 - y) - epsilon2*x)
    return [dx, dy]

In [ ]:
def plot_model_B(gamma=1.0, epsilon1=0.2, epsilon2=0.2, l1=0.3, l2=0.3):
    t_span = (0, 200)
    t_eval = np.linspace(*t_span, 2000)

    N0 = [0.4, 0.6]
    sol = solve_ivp(system_B, t_span, N0, t_eval=t_eval, args=(gamma, epsilon1, epsilon2, l1, l2))

    plt.figure(figsize=(12, 5))

    # (1) часова динаміка
    plt.subplot(1, 2, 1)
    plt.plot(sol.t, sol.y[0], label='x')
    plt.plot(sol.t, sol.y[1], label='y')
    plt.xlabel('t')
    plt.ylabel('Популяції')
    plt.title(f'Модель Б: γ={gamma}, ε₁={epsilon1}, ε₂={epsilon2}, l₁={l1}, l₂={l2}')
    plt.legend()
    plt.grid(True)

    # (2) фазовий портрет
    plt.subplot(1, 2, 2)
    x_vals = np.linspace(0, 1.2, 30)
    y_vals = np.linspace(0, 1.2, 30)
    X, Y = np.meshgrid(x_vals, y_vals)

    U = X*(X - l1)*(1 - X) - epsilon1*X*Y
    V = gamma*Y*((Y - l2)*(1 - Y) - epsilon2*X)

    plt.streamplot(X, Y, U, V, density=1.2, linewidth=1, arrowsize=1, color='black')

    # кілька траєкторій з різних початкових умов
    initials = [[0.05, 0.1], [0.1, 0.9], [0.3, 0.3], [0.5, 0.1], [0.8, 0.9], [1.0, 0.4]]
    for N0 in initials:
        sol = solve_ivp(system_B, t_span, N0, t_eval=t_eval, args=(gamma, epsilon1, epsilon2, l1, l2))
        plt.plot(sol.y[0], sol.y[1], lw=2)

    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Фазовий портрет')
    plt.xlim(0, 1.2)
    plt.ylim(0, 1.2)
    plt.grid(True)

    plt.tight_layout()
    plt.show()


# --- Інтерактивна панель ---
interact(
    plot_model_B,
    gamma=FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0),
    epsilon1=FloatSlider(min=0.0, max=0.5, step=0.05, value=0.2),
    epsilon2=FloatSlider(min=0.0, max=0.5, step=0.05, value=0.2),
    l1=FloatSlider(min=0.0, max=0.6, step=0.05, value=0.3),
    l2=FloatSlider(min=0.0, max=0.6, step=0.05, value=0.3)
)

interactive(children=(FloatSlider(value=1.0, description='gamma', max=2.0, min=0.5), FloatSlider(value=0.2, de…

<function __main__.plot_model_B(gamma=1.0, epsilon1=0.2, epsilon2=0.2, l1=0.3, l2=0.3)>

In [ ]:
from scipy.optimize import fsolve
import numpy as np

gamma = 1.0
epsilon1 = 0.2
epsilon2 = 0.2
l1 = 0.3
l2 = 0.3

def system_B_eqs(N, gamma, epsilon1, epsilon2, l1, l2):
    x, y = N
    dx = x*(x - l1)*(1 - x) - epsilon1*x*y
    dy = gamma*y*((y - l2)*(1 - y) - epsilon2*x)
    return [dx, dy]
# Якобі
def jacobian_B(x, y):
    J11 = (x - l1)*(1 - x) + x*(1 - 2*x + l1) - epsilon1*y
    J12 = -epsilon1*x
    J21 = gamma*y*(-epsilon2)
    J22 = gamma*((y - l2)*(1 - y) + y*(1 - 2*y + l2) - epsilon2*x)
    return np.array([[J11, J12],
                     [J21, J22]])
# список різних початкових наближень, щоб знайти усі можливі точки
guesses = [
    [0.0, 0.0],
    [0.1, 0.1],
    [0.5, 0.1],
    [0.1, 0.5],
    [0.8, 0.8],
    [1.0, 0.1],
    [0.1, 1.0],
    [1.0, 1.0]
]

solutions = []
for guess in guesses:
    sol = fsolve(system_B_eqs, guess, args=(gamma, epsilon1, epsilon2, l1, l2))
    sol = np.round(sol, 5)
    if all(sol >= 0) and all(sol <= 1.5):
        if not any(np.allclose(sol, s, atol=1e-4) for s in solutions):
            solutions.append(sol)

print("Особливі точки та їх типи:")
for s in solutions:
    x, y = s
    J = jacobian_B(x, y)
    eigvals = np.linalg.eigvals(J)
    real_parts = np.real(eigvals)

    print(f"\nТочка ({x:.4f}, {y:.4f})")
    print(f"  Власні числа: {eigvals}")

    if np.all(real_parts < 0):
        print("  → стійкий вузол або фокус")
    elif np.all(real_parts > 0):
        print("  → нестійкий вузол або фокус")
    elif real_parts[0] * real_parts[1] < 0:
        print("  → сідлова точка")
    else:
        print("  → центр або нейтральна стабільність")


Особливі точки та їх типи:

Точка (0.0000, 0.0000)
  Власні числа: [-0.3 -0.3]
  → стійкий вузол або фокус

Точка (0.3000, 0.0000)
  Власні числа: [ 0.21 -0.36]
  → сідлова точка

Точка (0.0000, 0.3000)
  Власні числа: [ 0.21 -0.36]
  → сідлова точка

Точка (0.6000, 0.6000)
  Власні числа: [ 0.18 -0.06]
  → сідлова точка

Точка (1.0000, 0.0000)
  Власні числа: [-0.7 -0.5]
  → стійкий вузол або фокус

Точка (0.0000, 1.0000)
  Власні числа: [-0.7 -0.5]
  → стійкий вузол або фокус
